# Identify Datasets

This notebook is a part of the [Crop Segmentation and Classification](https://github.com/planetlabs/notebooks/tree/master/jupyter-notebooks/crop-classification) notebook project. In this notebook, we identify and the ground truth data, define the area of intereste, and identify and download imagery data for use in crop segmentation and classification. We do this for two sets of data: a training set, where we will develop our segmentation and classification algorithms, and a testing set, where we test the accuracy of the resulting segmentation and classification.

The general sections are:
1. [Explore Ground Truth Data](#ground_truth)
2. [Identify Area of Interest](#aoi)
3. [Download Planet Scene](#pl_scene)

Ground truth crop type and boundary data is not easy to come by. Therefore, the area and time of interest for this problem is primarily defined by the availability of ground truth data. The [2015 Sacramento County Land Use DWR Survey Dataset](https://data.cnra.ca.gov/dataset/a6f3035f-0953-4b4b-bcbd-57c3b22cda28/resource/a287d32b-1f23-4aa0-83f5-3fe77c116e6b) is a free dataset covering Sacramento county in 2015. It provides hand-adjusted boundaries and provides crop types.

The primary satellite imagery we will use in this study is SSO Planetscope 2 imagery. We will use the [PSScene Analytic Product](https://developers.planet.com/docs/data/psscene/), which is 4-band (Blue, Green Red, Near-IR) and radiometrically corrected to at-sensor radiance. Correction to at-sensor radiance removes any variation in imagery between different satellites and allows for calculating vegetative indices. Further, the coefficients for correcting to at-sensor reflectance are provided in the scene metadata, which further improves the consistency between images taken at different times.

SSO Planetscope 2 satellites were launched Feb 14, 2017 ([news release](https://www.planet.com/pulse/planet-launches-satellite-constellation-to-image-the-whole-planet-daily/)), therefore they did not image Sacramento county in 2015. Although at this time we are focusing on PlanetScope imagery, in the future we may use Landsat 8 imagery as a bridge between 2015 and 2017.

### Usage Notes

To use this notebook, you will need an active [Planet API Key](https://www.planet.com/account/#/user-settings), and will need to make sure you're using [V2 of the Planet Python Client](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/).

In [1]:
# Notebook dependencies
import copy
from datetime import datetime
from functools import partial
import json
import os
import zipfile

import ipyleaflet as ipyl
import ipywidgets as ipyw
from IPython import display
import fiona
import pandas as pd
from planet import Auth, data_filter
from planet import Session
import pyproj
from pyproj import CRS, Transformer
import shapely
from shapely.geometry import shape, mapping
from shapely.ops import transform
import wget

<a id='ground_truth'></a>

## Explore Ground Truth Data

In this section we will download the ground truth data, filter it to crop features, and save it as a geojson dataset (geojson is our preferred format for processing in these notebooks).


#### Unzip data
We first start with the shapefile zip file downloaded from (https://data.cnra.ca.gov/dataset/a6f3035f-0953-4b4b-bcbd-57c3b22cda28/resource/a287d32b-1f23-4aa0-83f5-3fe77c116e6b/download/i15_crop_mapping_delta_2015_shp.zip).

To download this file yourself, in the `pre-data` directory, run:
```
$> wget https://water.ca.gov/-/media/DWR-Website/Web-Pages/Programs/Water-Use-And-Efficiency/Land-And-Water-Use/Land-Use-Surveys/Files/2015/15sa.zip
```

In [2]:
# To download this file yourself, in the `pre-data` directory, run:
url = 'https://data.cnra.ca.gov/dataset/a6f3035f-0953-4b4b-bcbd-57c3b22cda28/resource/a287d32b-1f23-4aa0-83f5-3fe77c116e6b/download/i15_crop_mapping_delta_2015_shp.zip'
wget.download(url, 'pre-data')

  8% [.....                                                              ]  4874240 / 59923641

 17% [...........                                                        ] 10469376 / 59923641

 25% [.................                                                  ] 15474688 / 59923641

 33% [......................                                             ] 19881984 / 59923641

 41% [...........................                                        ] 24813568 / 59923641

...

 74% [.................................................                  ] 44392448 / 59923641

 83% [.......................................................            ] 49815552 / 59923641

 91% [.............................................................      ] 54853632 / 59923641

100% [...................................................................] 59923641 / 59923641

'pre-data/i15_crop_mapping_delta_2015_shp (1).zip'

In [3]:
data_dir = 'data'
predata_dir = 'pre-data'

shapefile_zip = os.path.join(predata_dir, '15sa.zip')
shapefile_dir = os.path.join(data_dir, 'dwr_survey')

In [4]:
with zipfile.ZipFile(shapefile_zip, 'r') as zip_ref:
    zip_ref.extractall(shapefile_dir)

In [5]:
# Specify the shapefile location and ensure it indeed exists
survey_shapefile = os.path.join(shapefile_dir, 'SA15.shp')
assert os.path.isfile(survey_shapefile)

#### Prepare data

The data is provided as a shapefile. It is easier to process the data as geojson. Therefore, we will convert the data to geojson. Additionally, the data contains polygons that aren't crops. Since we are only interested in crops, we will filter the data to only the crop polygons.

We will use [fiona](https://fiona.readthedocs.io/en/latest/) to load the shapefile, [shapely](https://shapely.readthedocs.io/en/stable/) to manage the geometries

##### Reproject to WGS84

What is the coordinate reference system for this dataset?

In [6]:
src_proj = fiona.open(survey_shapefile, 'r').crs['init']
print(src_proj)

epsg:26910


Turns out it is [EPSG:26910](http://spatialreference.org/ref/epsg/26910/). Geojson only supports [EPSG:4326](http://spatialreference.org/ref/epsg/4326/). We will need to reproject the shapes.

In [7]:
# define projection
# from shapely [docs](https://shapely.readthedocs.io/en/stable/manual.html#other-transformations)
def define_to_wkt_projection(dataset):
    """dataset is obtained from fiona.open(file)"""
    src_proj = dataset.crs['init']
    dst_proj = "EPSG:4326"
    
    project_to_wkt = Transformer.from_crs(CRS(src_proj), CRS(dst_proj), always_xy=True).transform

    return project_to_wkt

def project_feature(feat, projection_fcn):
    g1 = shape(feat['geometry'])
    g2 = transform(projection_fcn, g1)
    feat['geometry'] = mapping(g2)

##### Filter to agricultural classes

The survey data has attributes that provide the crop type. These attributes are described in a pdf distributed with the shapefile. It was unzipped along with the shapefile files and is located at `data/dwr_survey/09legend.pdf`.

We are interested in the agricultural classes. Class is specified by the 'CLASS1' attribute of the feature.

The agricultural class label and descriptions are:
- G: Grain and Hay Crops
- R: Rice
- F: Field Crops
- P: Pasture
- T: Truck, Nursery, and Berry Crops
- D: Deciduous Fruits and Nutes
- C: Citrus and Tropical
- V: Vineyards

In [8]:
# Class ids from dwr_survey/09legend.pdf
agg_classes = ['G', 'R', 'F', 'P', 'T', 'D', 'C', 'V']

def is_agricultural(feat):
    return feat['properties']['CLASS1'] in agg_classes

#### Finally: Load data

Load the ground truth data into a list of geojson features, filtering to only agricultural classes and projecting to wkt. Because this process takes a while, save the loaded features for later use.

In [9]:
def load_ground_truth(filename):
    features = []
    with fiona.open(filename) as survey_data:
        to_wkt_projection = define_to_wkt_projection(survey_data)
        for feat in survey_data:
            if is_agricultural(feat):
                project_feature(feat, to_wkt_projection)
                features.append(feat)
    return features

features = load_ground_truth(survey_shapefile)
print(len(features))

7429


<a id='aoi'></a>

## Identify Area of Interest

In this section we will identify an area of interest (aoi) for each study. 

Selection of the area of interest for our study is based on the following:
1. compact representation of many crop classes
2. availability of imagery
3. as large as possible but smaller than a planet image (to allow for in-scene analysis)

We visualize the ground truth data in an interactive window that allows definition of the aoi. Then we query the Planet API to determine the availability of imatery. After using this interactive visualization, we identified two AOIs, which are defined in the notebook as `aoi_test` and `aoi_train`. Next, we identify the Planet scene we want to download for our study.

### Criteria 1: compact representation of many crop classes

Let's start by identifying a region of compact representation of many crop classes. We will do so by drawing a box (the aoi) over the map of the crops and then displaying the number of unique classes represented in the box.

In [10]:
# Assign colors to classes
# colors determined using [colorbrewer2.org](http://colorbrewer2.org/#type=sequential&scheme=BuGn&n=3)
colors = ['#ffffd9','#edf8b1','#c7e9b4','#7fcdbb','#41b6c4','#1d91c0','#225ea8','#0c2c84']
class_colors = dict((a,c) for a,c in zip(agg_classes, colors))

def get_color(cls):
    return class_colors[cls]

In [11]:
# Create crop feature layer
feature_collection = {
    "type": "FeatureCollection",
    "features": features
}

for f in feature_collection['features']:
    feature_color = get_color(f['properties']['CLASS1'])
    f['properties']['style'] = {
        'weight': 0,
        'fillColor': feature_color,
        'fillOpacity': 1}

feature_layer = ipyl.GeoJSON(data=feature_collection)

In [12]:
# Initialize map using parameters from above map
# and deleting map instance if it exists
try:
    del aoi_map
except NameError:
    pass


zoom = 11
center = [38.3586252, -121.3853994] # lat/lon

In [13]:
# Create map, adding box drawing controls
# Reuse parameters if map already exists
try:
    center = aoi_map.center
    zoom = aoi_map.zoom
except NameError:
    pass

# Change tile layer to one that makes it easier to see crop features
# Layer selected using https://leaflet-extras.github.io/leaflet-providers/preview/
map_tiles = ipyl.TileLayer(url='http://{s}.basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png')
aoi_map = ipyl.Map(
        center=center, 
        zoom=zoom,
        default_tiles = map_tiles
    )

aoi_map.add_layer(feature_layer)  

# Add box drawing control
# refs:
# https://github.com/kscottz/PythonFromSpace/blob/master/TheBasics.ipynb
# https://github.com/ellisonbg/ipyleaflet/blob/master/examples/DrawControl.ipynb
rectangle = {'shapeOptions': {'color': 'blue'}} 
dc = ipyl.DrawControl(
    polygon={}, polyline={}, # disable polygons and polylines
    rectangle={'shapeOptions': {'color': 'blue'}}
)

# When a box is drawn, update the label with the number of unique classes
# and save the box geometry as AOI
total_unique_classes = len(set([f['properties']['CLASS1'] for f in features]))
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

aois = []
def handle_draw(self, action, geo_json):
    if action == 'created':
        box_shape = shape(geo_json['geometry'])
        contained_features = [f for f in features
                              if shape(f['geometry']).within(box_shape)]
        unique_classes = set([f['properties']['CLASS1'] for f in contained_features])
        label.value = '{} unique classes out of {} total'.format(
            len(unique_classes), total_unique_classes)
        aois.append(geo_json)
    elif action == 'deleted':
        aois.remove(geo_json)
    
dc.on_draw(handle_draw)
aoi_map.add_control(dc) 

# Display map and label
ipyw.VBox([aoi_map, label])

From this map, we have identified two potential aois for a training dataset and a testing dataset

In [14]:
# Run this to use cached aois
aoi_train = {u'type': u'Polygon', u'coordinates': [[[-121.58460974693298, 38.29170496647727], [-121.58460974693298, 38.32726528409606], [-121.5248715877533, 38.32726528409606], [-121.5248715877533, 38.29170496647727], [-121.58460974693298, 38.29170496647727]]]}
aoi_test = {u'type': u'Polygon', u'coordinates': [[[-121.3113248348236, 38.28911976564886], [-121.3113248348236, 38.34622533958], [-121.2344205379486, 38.34622533958], [-121.2344205379486, 38.28911976564886], [-121.3113248348236, 38.28911976564886]]]}

In [15]:
aoi_train = {"type": "Polygon",
        "coordinates": [
          [
            [
              -121.58460974693298,
              38.29170496647727
            ],
            [
              -121.58460974693298,
              38.32726528409606
            ],
            [
              -121.5248715877533,
              38.32726528409606
            ],
            [
              -121.5248715877533,
              38.29170496647727
            ],
            [
              -121.58460974693298,
              38.29170496647727
            ]
          ]
        ]
      }

In [16]:
aoi_test = {"type":"Polygon",
            "coordinates":[
               [
                  [
                     -121.3113248348236,
                     38.28911976564886
                  ],
                  [
                     -121.3113248348236,
                     38.34622533958
                  ],
                  [
                     -121.2344205379486,
                     38.34622533958
                  ],
                  [
                     -121.2344205379486,
                     38.28911976564886
                  ],
                  [
                     -121.3113248348236,
                     38.28911976564886
                  ]
               ]
            ]
         }


### Criteria 2: availability of imagery

How many Planet images cover the AOI defined above?

We will answer this question by querying the Planet Data API, using the Planet Python Client.

[Planet Client Documentation](https://planet-sdk-for-python-v2.readthedocs.io/en/latest/) 

Much of this code is pulled from [PythonFromSpace/TheBasics.ipynb](https://github.com/kscottz/PythonFromSpace/blob/master/TheBasics.ipynb)

In [17]:
# if your Planet API Key is not set as an environment variable, you can paste it below
if 'PL_API_KEY' in os.environ:
    API_KEY = os.environ['PL_API_KEY']
else:
    API_KEY = 'PASTE_API_KEY_HERE'
    os.environ['PL_API_KEY'] = API_KEY

client = Auth.from_key(API_KEY)

#### Query Planet API

Filter to scenes that contain AOI. If the number is zero, go back and redefine the AOI to be smaller.

In [18]:
# create an API Request from the search specifications for data

item_type = ['PSScene']

geom_filter_test = data_filter.geometry_filter(aoi_test)
geom_filter_train = data_filter.geometry_filter(aoi_train)
date_range_filter = data_filter.date_range_filter("acquired", gt=datetime(month=6, day=1, year=2016), lt=datetime(month=10, day=1, year=2016))
cloud_cover_filter = data_filter.range_filter('cloud_cover', lt=0.10)

combined_filter_test = data_filter.and_filter([geom_filter_test, date_range_filter, cloud_cover_filter])
combined_filter_train = data_filter.and_filter([geom_filter_train, date_range_filter, cloud_cover_filter])
    
# Run a quick search for our TRAIN data
async with Session() as sess:
    cl = sess.client('data')
    results = cl.search(name='temp_search_train',search_filter=combined_filter_train, item_types=item_type)
    train_result_list = [i async for i in results]
    
# Run a quick search for our TEST data
async with Session() as sess:
    cl = sess.client('data')
    results = cl.search(name='temp_search_test',search_filter=combined_filter_test, item_types=item_type)
    test_result_list = [i async for i in results]

Let's look at our search results:

In [19]:
print("Number of train scene results: ", len(train_result_list))
print("Number of test scene results: ", len(test_result_list))

Number of train scene results:  51
Number of test scene results:  53


That's a lot of results! However, we need to make sure that images we use for our analysis will cover most of our AOI. We do not currently have API support for a `coverage` feature, but we can build this function below!

In [20]:
# build a coverage filter to ensure we are getting images that cover most of our AOI!
def determine_overlaps(search_aoi, result_list):
    scenes = []
    planet_map = {}
    for item in result_list:
        planet_map[item['id']]=item
        props = item['properties']
        props["id"] = item['id']
        props["geometry"] = item["geometry"]
        props["thumbnail"] = item["_links"]["thumbnail"]
        scenes.append(props)
    scenes = pd.DataFrame(data=scenes)
    
    aoi_shape = shape(search_aoi)
        
    footprints = []
    overlaps = []
    # go through the geometry from our api call, convert to a shape and calculate overlap area.
    # also save the shape for safe keeping
    for footprint in scenes["geometry"].tolist():
        s = shape(footprint)
        footprints.append(s)
        overlap = 100.0*(aoi_shape.intersection(s).area / aoi_shape.area)
        overlaps.append(overlap)

    # take our lists and add them back to our dataframe
    scenes['overlap'] = pd.Series(overlaps, index=scenes.index)
    scenes['footprint'] = pd.Series(footprints, index=scenes.index)

    
    near_full_coverage = scenes["overlap"] > 95    
    good_scenes = scenes[(near_full_coverage)]
    return good_scenes

scenes_train = determine_overlaps(aoi_train, train_result_list)
print("Number of near-full coverage train scene results: ", len(scenes_train))

scenes_test = determine_overlaps(aoi_test, test_result_list)
print("Number of near-full coverage test scene results: ", len(scenes_test))

Number of near-full coverage train scene results:  6
Number of near-full coverage test scene results:  2


##### Select Scenes

That helps us narrow our images by a lot! 

Now we can explore these resulting images. Feel free to use the tables below to narrow which images you will use. You can also grab the image_ids (`scenes_test['id']`) and put them into Planet Explorer to see what these images look like. 

In [21]:
scenes_test

,acquired,anomalous_pixels,clear_confidence_percent,clear_percent,cloud_cover,cloud_percent,ground_control,gsd,heavy_haze_percent,instrument,...,sun_elevation,updated,view_angle,visible_confidence_percent,visible_percent,id,geometry,thumbnail,overlap,footprint
15,2016-08-31T18:02:57.353393Z,0,100.0,100.0,0.0,0.0,True,4.1,0.0,PS2,...,49.1,2021-01-27T17:32:34Z,3.2,81.0,100.0,20160831_180257_0e26,"{'coordinates': [[[-121.47564640227792, 38.394...",https://tiles.planet.com/data/v1/item-types/PS...,100.000000,POLYGON ((-121.47564640227792 38.3942146960691...
48,2016-07-14T18:00:05.23729Z,0,100.0,100.0,0.0,0.0,True,4.0,0.0,PS2,...,57.3,2021-01-27T10:57:23Z,1.7,100.0,100.0,20160714_180005_0e26,"{'coordinates': [[[-121.45511358606822, 38.388...",https://tiles.planet.com/data/v1/item-types/PS...,98.810225,POLYGON ((-121.45511358606822 38.3881090430578...


In [22]:
scenes_train

,acquired,anomalous_pixels,cloud_cover,ground_control,gsd,instrument,item_type,pixel_resolution,provider,published,...,clear_percent,cloud_percent,heavy_haze_percent,light_haze_percent,shadow_percent,snow_ice_percent,visible_confidence_percent,visible_percent,overlap,footprint
3,2016-09-30T18:03:55.671689Z,0,0.0,True,4.0,PS2,PSScene,3,planetscope,2021-01-27T22:26:55Z,...,100.0,0.0,0.0,0.0,0.0,0.0,76.0,100.0,97.263954,POLYGON ((-121.71327278499317 38.3992103520006...
16,2016-08-31T18:02:31.641717Z,0,0.0,True,4.1,PS2,PSScene,3,planetscope,2021-01-27T17:32:22Z,...,100.0,0.0,0.0,0.0,0.0,0.0,77.0,100.0,100.000000,POLYGON ((-121.67803255865208 38.3633227859237...
22,2016-08-22T18:12:11.330979Z,0,0.0,True,3.2,PS2,PSScene,3,planetscope,2021-01-27T15:52:01Z,...,100.0,0.0,0.0,0.0,0.0,0.0,98.0,100.0,99.544944,POLYGON ((-121.57834174857301 38.3744768573394...
38,2016-08-09T18:02:40.705088Z,0,0.0,True,3.9,PS2,PSScene,3,planetscope,2021-01-27T14:21:13Z,...,100.0,0.0,0.0,0.0,0.0,0.0,77.0,100.0,100.000000,POLYGON ((-121.73130158017905 38.3825463300213...
39,2016-08-08T00:05:47.036676Z,0,0.0,True,3.1,PS2,PSScene,3,planetscope,2021-01-27T13:52:32Z,...,100.0,0.0,0.0,0.0,0.0,0.0,99.0,100.0,98.570159,POLYGON ((-121.47610527998044 38.2034291825701...
44,2016-07-17T15:32:12.544923Z,0,0.0,True,3.2,PS2,PSScene,3,planetscope,2021-01-27T11:37:53Z,...,100.0,0.0,0.0,0.0,0.0,0.0,99.0,100.0,96.468523,POLYGON ((-121.7197718974563 38.23933282380383...


For the purposes of this example, we've picked two scenes below:

In [23]:
# Train Scene
train_thumbnail = 'https://tiles.planet.com/data/v1/item-types/PSScene/items/20160831_180231_0e0e/thumb?api_key={}'.format(API_KEY)
display.Image(url=train_thumbnail)

In [24]:
# Test Scene
test_thumbnail = 'https://tiles.planet.com/data/v1/item-types/PSScene/items/20160831_180257_0e26/thumb?api_key={}'.format(API_KEY)
display.Image(url=test_thumbnail)

<a id='pl_scene'></a>

## Save datasets

We will save the train and test AOI and the ground truth data.

In [25]:
def save_geojson(features, filename):
    with open(filename, "w") as f:
        f.write(json.dumps(features))

In [26]:
# save train AOI
save_geojson(aoi_train, os.path.join(predata_dir, 'aoi-train.geojson'))

# save test
save_geojson(aoi_test, os.path.join(predata_dir, 'aoi-test.geojson'))

# save ground truth data
save_geojson(features, os.path.join(predata_dir, 'ground-truth.geojson'))